In [1]:
import pickle,os
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import scipy.signal as signal
import matplotlib.gridspec as gridspec
from itertools import chain
#import seaborn as sns
import networkx as nx
#sns.reset_defaults()

In [16]:
pathA="processedData/alcohol/"
channel=pickle.load(open("pickles/channel.pickle","rb"))
locations=pickle.load(open("pickles/locations.pickle","rb"))
threshold=0.7

cluster=OrderedDict()
cluster['C']=['C{}'.format(i) for i in range(1,7)]
cluster['F']=['F{}'.format(i) for i in range(1,9)]
cluster['O']=['O{}'.format(i) for i in range(1,3)]
cluster['P']=['P{}'.format(i) for i in range(1,9)]

cluster['C'].extend(['CZ'])
cluster['F'].extend(['FZ'])
cluster['O'].extend(['OZ'])
cluster['P'].extend(['PZ'])


cluster['T']=['T7','T8']

studiedSensors=list(cluster.keys())

In [30]:
cluster

OrderedDict([('C', ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CZ']),
             ('F', ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FZ']),
             ('O', ['O1', 'O2', 'OZ']),
             ('P', ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'PZ']),
             ('T', ['T7', 'T8'])])

In [17]:
sensorMap={}
sensorMapRev={}
count=0
for sensor in channel:
    sensorMap[sensor]=count
    sensorMapRev[count]=sensor
    count+=1 

In [18]:
def initConnectionMatrix():
    matrix={}
    for i in range(len(cluster)):
        for j in range(0,i+1):
            key=studiedSensors[i]+"-"+studiedSensors[j]
            matrix[key]=0
    return matrix

def validateSensors(i,j):
    sensorA=sensorMapRev[i]
    sensorB=sensorMapRev[j]
    if sensorA[0] in studiedSensors and sensorB[0] in studiedSensors:
        if sensorA in cluster[sensorA[0]] and sensorB in cluster[sensorB[0]]:
            return True
        else:
            return False
    else:
            return False
        
def addConnection(trail,stimulus,i,j):
    sensorA=sensorMapRev[i]
    sensorB=sensorMapRev[j]
    try:
        key=sensorA[0]+'-'+sensorB[0]
        sumConnections[stimulus][key]+=1
        trailMatrix[trail][key]+=1
    except:
        key=sensorB[0]+'-'+sensorA[0]
        sumConnections[stimulus][key]+=1  
        trailMatrix[trail][key]+=1
        
def savitzkyGolay(data, parameters):
    #print(parameters)
    smooth_data = signal.savgol_filter(data, parameters[0], parameters[1])
    peel = np.array(data[-len(smooth_data):]) - np.array(smooth_data)
    return smooth_data, peel

In [19]:
sumConnections={}
stimulusFreq={}
stimuli=['S1obj','S2match','S2nomatch']
for stimulus in stimuli:
    sumConnections[stimulus]=initConnectionMatrix()
    stimulusFreq[stimulus]=0

In [20]:

trails=os.listdir(pathA)
clusterMatrix=initConnectionMatrix()

data=set()
trailMatrix=OrderedDict()

for trail in trails:
    patient,stimulus=chain(trail.split("_"))
    stimulusFreq[stimulus]+=1
    data=pickle.load(open(pathA+trail,"rb"))
    tempdata=[]
    trailMatrix[trail]=initConnectionMatrix()
    for sensor in channel:
        tempdata.append(data[sensor])
    corrMatrix=np.corrcoef(tempdata)
    
    for i in range(len(tempdata)):
        for j in range(0,i):
            if validateSensors(i,j):
                if corrMatrix[i][j]>=threshold:
                    addConnection(trail,stimulus,i,j)      

c:\program files (x86)\python36-32\lib\site-packages\numpy\lib\function_base.py:3167: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\program files (x86)\python36-32\lib\site-packages\numpy\lib\function_base.py:3168: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [21]:
stimulusFreq

{'S1obj': 77, 'S2match': 77, 'S2nomatch': 77}

In [22]:
zMatrix=initConnectionMatrix()
activation={}
for stimulus in stimuli:
    activation[stimulus]=initConnectionMatrix()
    
for conn in zMatrix:
    if conn[0]==conn[-1]:
        l=len(cluster[conn[0]])
        zMatrix[conn]=(l*(l-1))/2
    else:
        zMatrix[conn]=len(cluster[conn[0]])*len(cluster[conn[-1]])

        
for stimulus in stimuli:
    n=stimulusFreq[stimulus]
    for conn in sumConnections[stimulus]:
        z=zMatrix[conn]
        value=sumConnections[stimulus][conn]
        activation[stimulus][conn]=value/(z*n)
        if stimulus=="S1obj" and conn=="C-C":
            print(value,z,n)

403 21.0 77


In [23]:
pickle.dump(activation,open("activation/waveAndpeel/alcoholActivation.pickle","wb"))
#pickle.dump(activation,open("activation/alcoholActivation.pickle","wb"))

In [24]:
activation

{'S1obj': {'C-C': 0.24922696351267778,
  'F-C': 0.14491857349000206,
  'F-F': 0.4531024531024531,
  'O-C': 0.10265924551638837,
  'O-F': 0.0707070707070707,
  'O-O': 0.7619047619047619,
  'P-C': 0.14801071943929087,
  'P-F': 0.06573673240339907,
  'P-O': 0.5858585858585859,
  'P-P': 0.5562770562770563,
  'T-C': 0.24953617810760667,
  'T-F': 0.14935064935064934,
  'T-O': 0.11688311688311688,
  'T-P': 0.16883116883116883,
  'T-T': 0.11688311688311688},
 'S2match': {'C-C': 0.274582560296846,
  'F-C': 0.1686250257678829,
  'F-F': 0.5678210678210678,
  'O-C': 0.16573902288188003,
  'O-F': 0.09331409331409331,
  'O-O': 0.8311688311688312,
  'P-C': 0.21232735518449805,
  'P-F': 0.08128908128908129,
  'P-O': 0.683982683982684,
  'P-P': 0.6201298701298701,
  'T-C': 0.2820037105751391,
  'T-F': 0.1673881673881674,
  'T-O': 0.22510822510822512,
  'T-P': 0.24025974025974026,
  'T-T': 0.22077922077922077},
 'S2nomatch': {'C-C': 0.22139764996907854,
  'F-C': 0.11523397237682952,
  'F-F': 0.517316017

In [25]:
t={}
for trails in trailMatrix:
    trail,stimulus=chain(trails.split('_'))
    if stimulus not in t:
        t[stimulus]={}
    for cons in trailMatrix[trails]:
        if cons not in t[stimulus]:
            t[stimulus][cons]=[]
        t[stimulus][cons].append(trailMatrix[trails][cons]/zMatrix[cons])

In [26]:
activationStd={}
for stimulus in stimuli:
    activationStd[stimulus]=initConnectionMatrix()

for stimulus in t:
    for cons in t[stimulus]:
        activation[stimulus][cons]=np.mean(t[stimulus][cons])
        activationStd[stimulus][cons]=np.std(t[stimulus][cons]) 

In [27]:
activationStd
#pickle.dump(activationStd,open("activation/alcoholActivationStd.pickle","wb"))
pickle.dump(activationStd,open("activation/waveAndPeel/alcoholActivationStd.pickle","wb"))

In [15]:
activationStd

{'S1obj': {'C-C': 0.2549203828385932,
  'F-C': 0.26884390842670092,
  'F-F': 0.29574181679884559,
  'O-C': 0.29743679615227397,
  'O-F': 0.25869778149999162,
  'O-O': 0.31604696534102128,
  'P-C': 0.26464418129469197,
  'P-F': 0.22908744475528697,
  'P-O': 0.29665058807805322,
  'P-P': 0.29802084906724929,
  'T-C': 0.27296696647022528,
  'T-F': 0.29005944259474875,
  'T-O': 0.36252665141202312,
  'T-P': 0.29485643350204493,
  'T-T': 0.48104569292083466},
 'S2match': {'C-C': 0.25458109136178703,
  'F-C': 0.28175918639800218,
  'F-F': 0.24877340692305538,
  'O-C': 0.3098780760637192,
  'O-F': 0.31376129984465601,
  'O-O': 0.26103573041872435,
  'P-C': 0.28005534737524257,
  'P-F': 0.30129253088735836,
  'P-O': 0.23903933394223886,
  'P-P': 0.24083592494103215,
  'T-C': 0.26395011353291919,
  'T-F': 0.32858159976804457,
  'T-O': 0.39317751358128278,
  'T-P': 0.33948276561407509,
  'T-T': 0.49995783259532872},
 'S2nomatch': {'C-C': 0.19799974159569053,
  'F-C': 0.23901788159171136,
  'F-F'

In [43]:
activation

{'S1obj': {'C-C': 0.21212121212121213,
  'F-C': 0.11152339723768295,
  'F-F': 0.39826839826839827,
  'O-C': 0.07050092764378478,
  'O-F': 0.06445406445406446,
  'O-O': 0.7272727272727273,
  'P-C': 0.11069882498453927,
  'P-F': 0.05723905723905724,
  'P-O': 0.49446849446849445,
  'P-P': 0.474025974025974,
  'T-C': 0.19202226345083487,
  'T-F': 0.11183261183261184,
  'T-O': 0.08441558441558442,
  'T-P': 0.12626262626262627,
  'T-T': 0.09090909090909091},
 'S2match': {'C-C': 0.23685837971552257,
  'F-C': 0.13234384662956092,
  'F-F': 0.5050505050505051,
  'O-C': 0.13852813852813853,
  'O-F': 0.0735930735930736,
  'O-O': 0.8051948051948052,
  'P-C': 0.17233560090702948,
  'P-F': 0.06846240179573512,
  'P-O': 0.5916305916305916,
  'P-P': 0.5432900432900433,
  'T-C': 0.24118738404452691,
  'T-F': 0.12193362193362194,
  'T-O': 0.16883116883116883,
  'T-P': 0.18037518037518038,
  'T-T': 0.14285714285714285},
 'S2nomatch': {'C-C': 0.18119975262832405,
  'F-C': 0.07503607503607504,
  'F-F': 0.45